In [2]:
import numpy as np 
import pandas as pd 
import os
from keras_callbacks import *
from keras.layers import *
from keras.preprocessing import text, sequence
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import *
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import keras
from keras import backend as K
from random import randint
from keras.engine.topology import Layer
from gensim.models import KeyedVectors
from train_config import targets
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
K.clear_session()

处理数据，整理成list

In [3]:
file = ['datasets/train_data_jieba.txt', 'datasets/valid_data_jieba.txt', 'datasets/test_data_jieba.txt']
X_train = []
X_valid = []
X_test = []

for f_name in file:
    with open(f_name,'r',encoding='utf-8') as F:
        if f_name == 'datasets/train_data_jieba.txt':
            line = F.readline()
            while line:
                X_train.append(line)
                line = F.readline()
                
        elif f_name == 'datasets/valid_data_jieba.txt':
            line = F.readline()
            while line:
                X_valid.append(line)
                line = F.readline()
                
        elif f_name == 'datasets/test_data_jieba.txt':
            line = F.readline()
            while line:
                X_test.append(line)
                line = F.readline()

处理labels

In [4]:
name = ['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']
train_data = pd.read_csv('datasets/sentiment_analysis_trainingset.csv')
valid_data = pd.read_csv('datasets/sentiment_analysis_validationset.csv')

y_train = []
y_valid = []
for N in name:
    Y_train = train_data[[N]]
    Y_train = np.array(Y_train)+2
    Y_train = to_categorical(Y_train, num_classes=4)
    y_train.append(Y_train)
    
    Y_valid = valid_data[[N]]
    Y_valid = np.array(Y_valid)+2
    Y_valid = to_categorical(Y_valid, num_classes=4)
    y_valid.append(Y_valid)

In [5]:
print(len(y_train[0]))
print(len(y_valid))
print(y_train[3])

105000
20
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
EMBEDDING_FILE = 'datasets/word2vec_400v'
# 词典大小，embedding矩阵大小
max_features=170000

# 最长句子
maxlen=1024
# 词向量维度
embed_size=400

In [7]:
tok=text.Tokenizer(num_words=max_features,lower=False)
tok.fit_on_texts((X_train + X_valid + X_test))
train = tok.texts_to_sequences(X_train)
valid = tok.texts_to_sequences(X_valid)
test = tok.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(train, maxlen=maxlen, padding='pre')
x_valid = sequence.pad_sequences(valid, maxlen=maxlen, padding='pre')
x_test = sequence.pad_sequences(test, maxlen=maxlen, padding='pre')

In [8]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [9]:
word_index = tok.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [11]:
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            trainable=False)

In [12]:
def dp_block(inputs):
    
    conv = Conv1D(filters=128, strides=1, kernel_size=3, activation='relu')(inputs)  # keep dimension the same
    pool_avg = AveragePooling1D(pool_size=3, strides=2, padding='valid')(conv)
    pool_max = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv)
#     conv = Conv1D(filters=128, strides=1, kernel_size=3, activation='relu')(conv)
#     shortcut = Concatenate()([conv, pool])
    output = Add()([pool_avg, pool_max])
    return output

In [13]:
def attention(inputs):
    att = Dense(1, activation='sigmoid')(inputs)
    weight = Reshape((-1,))(att)
    weight = RepeatVector(128)(weight)
    weight = Permute((2,1))(weight)
    mul = Multiply()([inputs, weight])
#     bn = BatchNormalization()(mul)
    
    return mul

In [14]:
def build_model(embedding_layer, maxlen, targets):
    sent_input = Input((maxlen,), dtype='int32')
    sent_embedded = embedding_layer(sent_input)
    output_list_1 = []
    
    conv_3 = Conv1D(filters=256, strides=1, kernel_size=3, padding='same', activation='relu')(sent_embedded)
    conv_3 = attention(conv_3)
    conv_3 = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_3)
    
    conv_2 = Conv1D(filters=256, strides=1, kernel_size=2, padding='same', activation='relu')(sent_embedded)
    conv_2 = attention(conv_2)
    conv_2 = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_2)
    
    conv_4 = Conv1D(filters=256, strides=1, kernel_size=4, padding='same', activation='relu')(sent_embedded)
    conv_4 = attention(conv_4)
    conv_4 = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_4)
    
    conv_5 = Conv1D(filters=256, strides=1, kernel_size=5, padding='same', activation='relu')(sent_embedded)
    conv_5 = attention(conv_5)
    conv_5 = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_5)
    
    conv = Concatenate()([conv_2, conv_3, conv_4, conv_5])
#     conv_att = dp_block(conv)
    conv_1 = Conv1D(256, strides=1, kernel_size=3, activation='relu')(conv)
    pool_avg = AveragePooling1D(pool_size=3, strides=2, padding='valid')(conv_1)
    pool_max = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_1)
    pool = Add()([pool_avg, pool_max])
    
    conv_1 = Conv1D(128, strides=1, kernel_size=3, activation='relu')(conv_1)
    pool_avg = AveragePooling1D(pool_size=3, strides=2, padding='valid')(conv_1)
    pool_max = MaxPooling1D(pool_size=3, strides=2, padding='valid')(conv_1)
    pool = Add()([pool_avg, pool_max])
    
    for target in targets:
        
        flat_1 = Flatten(name='flatten_1'+target)(pool)
        if target in ['location_distance_from_business_district', 'dish_look', 'service_wait_time', ]:
            flat_2 = Flatten(name='flatten_2'+target)(conv)
            flat_1 = concatenate([flat_1, flat_2])
            flat_1 = Dense(64, activation='relu', name='Dense_0_'+target)(flat_1)
            
        dense = Dense(32, activation='relu', name='Dense_1_'+target)(flat_1)
        output_target = Dense(4, name='output_'+target, activation='softmax')(dense)
        output_list_1.append(output_target)
    
    
    model = Model(sent_input, output_list_1)
    return model

In [15]:
batch_size = 32
epochs = 2

In [16]:
model_name = 'DPCNN'
lr_schedule = generate_learning_rate_schedule(0.0001, 0.003, 8, 0)
checkpoint = generate_check_point(model_name)
early_stopping = generate_early_stopping()
tensorboard = generate_tensorboard(model_name, 'ALL')
callbacks=[lr_schedule, checkpoint, early_stopping, tensorboard]

In [17]:

model = build_model(embedding_layer, maxlen, targets)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1024, 400)    68000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 1024, 128)    102528      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1024, 128)    153728      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [ ]:
for i in range(10):
    model.optimizer = Adam(lr=0.003/(i+1))
    model.fit(x_train,
              y_train, 
              batch_size=batch_size,
              epochs=1)
    preds = model.predict(x_valid, batch_size=32, verbose=True)
    score = []
    for i in range(20):
        score.append(f1_score(np.argmax(np.array(y_valid[i]), axis=1), np.argmax(np.array(preds[i]), axis=1), average='macro'))
    print(score)
    print(np.mean(score))

Epoch 1/1
15000/15000 [==============================] - 41s 3ms/step
[0.44089434582611275, 0.2580991016658746, 0.40672554997240773, 0.2343910472075645, 0.5886311475253204, 0.241788886593679, 0.2327158889325668, 0.5169092689825604, 0.4448544986070259, 0.49551893724994284, 0.45953580616870554, 0.37545487079667317, 0.34225006328325647, 0.4139932027489085, 0.4129688215614806, 0.4853359928053611, 0.2304936726463635, 0.3420486031536833, 0.48935650465264163, 0.5818451043913774]
0.3996905657385753
Epoch 1/1


D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


15000/15000 [==============================] - 40s 3ms/step
[0.4384944929820494, 0.2931268270864748, 0.418240518500119, 0.2594790707667198, 0.6688000396041506, 0.241788886593679, 0.49081208401914656, 0.6046356987105483, 0.48665197086129536, 0.5022054660935977, 0.5084081789727279, 0.4864529495383222, 0.5132241768345998, 0.579144805235693, 0.5637735693560045, 0.5697540538019062, 0.26702170853133445, 0.5141239345513239, 0.5315748414011539, 0.5937432180796737]
0.476572824576026
Epoch 1/1
15000/15000 [==============================] - 40s 3ms/step
[0.4466385212638041, 0.3478608548105978, 0.4969272046045684, 0.32515209261970934, 0.6879627640779828, 0.24176221464724718, 0.5451510587170967, 0.6127733994623398, 0.5287547800838409, 0.5423733658751313, 0.49224248233803947, 0.46077247719308634, 0.48699668731566187, 0.5612551236231849, 0.5829899455799297, 0.5131087577321644, 0.31077352025739474, 0.5357306982186018, 0.5290652553330534, 0.5927757259101984]
0.4920533464831817
Epoch 1/1
15000/15000 [==

In [71]:
y = np.argmax(np.array(preds), axis=2)-2
print(len(y))
test_data = pd.read_csv('datasets/sentiment_analysis_testa.csv')
i=0
for index in name:
    test_data.loc[:,index] = y[i]
    i=i+1
print(y[0][:100])

[-2 -2  1  1 -2  1 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2  1 -2 -2 -2 -2 -2
 -2 -2  1  1 -2 -2 -2 -2  1  1  1  1 -2  1 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2  1  1  1 -2  1 -2  1 -2 -2 -2 -2  1 -2  1 -2
 -2 -2 -2 -2]


In [69]:
test_data.to_csv('datasets/results.csv',encoding='utf-8',index=False)

In [70]:
test_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",-2,-2,-2,-2,1,-2,-2,0,...,1,-2,1,1,1,1,-2,-2,1,-2
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-1,0,-2,-2,0,-2
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",1,1,-2,-2,-1,-2,-2,0,...,-2,-2,-2,-2,-2,0,-2,-2,-1,-1
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",1,-2,-2,-2,1,-2,-2,-2,...,1,-2,-2,-2,-2,1,-2,1,1,1
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",-2,-2,-2,-2,1,-2,-2,-2,...,1,1,-2,-2,-2,1,-2,-2,1,-2
